In [2]:
__author__ = 'T. Sánchez-Pastor'
__date__   = '19 de Julio de 2021'
# Modules
import numpy as np
import matplotlib.pyplot as plt
from math import pi, sqrt
from scipy.special import gamma, hyp2f1
import os
from numba import njit, prange
from atomic_units import ao, vo, e, hbar, me, Eh, to

The energy spectrum solution for a two identical-ultracold atoms in completely anisotropic traps is:

$$J_{3D} = 4 \pi \left[ W_{3D}(E) + \int^\infty_0 d\beta\  I_{3D}(E, \beta)\right]$$

with $J_{3D} = \frac{1}{a_{3D}}$. The functions $W_{3D}$ and $I_{3D}$ are defined as follows:

$$W_{3D} = -\frac{\pi}{2} \sqrt{\frac{\eta_x \eta_y \eta_z}{2}} \sum_{(n_x, n_y) \in C^{3D}_E} \left[ \frac{2^{n_x+n_y-1} \Gamma\left(\frac{1}{4} - \frac{E - \epsilon_{n_x} - \epsilon_{n_y}}{2}\right)}{\Gamma \left(\frac{1-n_x}{2}\right)^2 \Gamma\left(\frac{1-n_y}{2}\right)^2 \Gamma\left(n_x + 1\right) \Gamma\left(n_y + 1\right) \Gamma\left(\frac{3}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}\right)}    \right],$$

$$ I_{3D} = -e^{\beta E} \prod_{\alpha=x,y,z}\sqrt{\frac{\eta_\alpha}{4\pi \sinh(\eta_\alpha\beta)}} + \left(\frac{1}{4\pi \beta}\right)^{3/2} + \sqrt{\frac{\pi\eta_x \eta_z}{8\sinh(\beta)}} \sum_{(n_x, n_y) \in C^{3D}_E} \left[ \frac{2^{n_x + n_y - 1/2} e^{\beta(E - \epsilon_{n_x} - \epsilon_{n_y})}}{\Gamma \left(\frac{1-n_x}{2}\right)^2 \Gamma\left(\frac{1-n_y}{2}\right)^2 \Gamma\left(n_x + 1\right) \Gamma\left(n_y + 1\right)} \right]$$

where, $\epsilon_{n_j} = \eta_j(n_j + 1/2)$, $\eta_j$ the anisotropies with respect the constant axis and $C^{3D}_E : \left\{ (n_x, n_y)|n_{x,y} = 0, 2, 4, 6, ...; \epsilon_{n_x} + \epsilon_{n_y} + 1/2 \leq E\right\}$.

There are some techniques which may speed up the calculation for the function $J_{3D}$. First,
$$J_{3D}(E) = 4\pi\left[ \int^\Lambda_0 A_{3D}d\beta + B^{(1)}_{3D}(E, \Lambda) + B^{(2)}_{3D}(E, \Lambda + \left(\frac{1}{2\pi} \right)^{3/2}\frac{1}{\sqrt{2\Lambda}}\right]$$

where $\Lambda$ is an arbitrary finite positive number, and the functions are defined as

$$A_{3D} = -e^{\beta E}\prod_\alpha \sqrt{\frac{\eta_\alpha}{4\pi \sinh(\eta_\alpha \beta)}} + \left(\frac{1}{4\pi \beta} \right)^{3/2};$$

$$B^{(1)}_{3D} = (-1) \sum_{(n_x, n_y) \in C^{3D}_E}\left\{ \frac{2^{n_x + n_y - 5/2} \sqrt{\pi \eta_x \eta_z}\ \Gamma\left(\frac{1}{4} - \frac{E - \epsilon_{n_x} - \epsilon_{n_y}}{2}\right) e^{(E - \epsilon_{n_x} - \epsilon_{n_y} - 3/2)\Lambda} }{\Gamma \left(\frac{1-n_x}{2}\right)^2 \Gamma\left(\frac{1-n_y}{2}\right)^2 \Gamma\left(n_x + 1\right) \Gamma\left(n_y + 1\right) \Gamma\left(\frac{5}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}\right)} \\ \sqrt{e^{2\Lambda} - 1} _2F^1 \left[1, \frac{3}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}, \frac{5}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}, e^{-2\Lambda} \right] \right\}$$

$$ B^{(2)}_{3D} = \sqrt{\pi \eta_x \eta_z cosch \Lambda} \sum_{(n_x, n_y) \in C^{3D}_E} \frac{2^{n_x + n_y - 1} e^{(E - \epsilon_{n_x} - \epsilon_{n_y} - 2)\Lambda} (e^{2\Lambda - 1}) _2F^1\left[1, \frac{3}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}, \frac{5}{4} - \frac{E- \epsilon_{n_x} - \epsilon_{n_y}}{2}, e^{-2\Lambda} \right] }{\Gamma \left(\frac{1-n_x}{2}\right)^2 \Gamma\left(\frac{1-n_y}{2}\right)^2 \Gamma\left(n_x + 1\right) \Gamma\left(n_y + 1\right) [2(E - \epsilon_{n_x} - \epsilon_{n_y}) - 1]} $$

In [8]:
# Parameters:
eta_x  = 1
eta_y  = 1
eta_z  = 0.1
nx     = 0
ny     = 0
nz     = 0
E      = -0.7660728376220698

In [9]:
# Functions:
def en(n, eta):
    return eta*(n + 1/2)

def A3D_int(a, b, h, eta_x, eta_y, eta_z, E):
    '''
    Chen et al. (2020)
    Parameters:
    -----------
    a:     left limit of the X-axis
    b:     right limit of the X-axis
    h:     step size
    nj:    levels
    eta_j: wj/wy
    E:     energy
    Lambda:cutoff
    
    Outputs:
    --------
    Value of the integral applying the trapezoidal integration method: step * f(t)
    '''
    out = 0
    N = int((2*b-2*a)/h)
    beta = np.arange(2*a, 2*b, h)
    for j in prange(N):
        out += -np.exp(beta[j]*E) * np.sqrt(eta_x*eta_y*eta_z/((4*pi)**3*np.sinh(eta_x*beta[j])*np.sinh(eta_y*beta[j])*np.sinh(eta_z*beta[j])))\
        + 1/(4*pi*beta[j])**(3/2)
    out*=h
    return out

def en(n, eta):
    return eta*(n + 1/2)

def W3D(nx, ny, nz, etax, etay, etaz, E):
    suma = 0
    for i in range(0, nx+1):
        for j in range(0, ny+1):
            if i%2==0 and j%2==0 and en(i, etax) + en(j, etay) + 1/2 <= E:
                suma += 2**(i + j - 1)*gamma(1/4 - (E - en(i, etax) - en(j, etay))/2)/\
                (gamma((1-i)/2)**2 * gamma((1-j)/2)**2 * gamma(1+i) * gamma(1+j) *\
                 gamma(3/4 - (E - en(i, etax) - en(j, etay))/2))
    return -pi/2 * sqrt(etax*etay*etaz/2) * suma

def B1_3D(nx, ny, nz, etax, etay, etaz, E, Lambda):
    suma = 0
    for i in range(0, nx+1):
        for j in range(0, ny+1):
            if i%2==0 and j%2==0 and en(i, etax) + en(j, etay) + 1/2 <= E:
                suma += 2**(i + j - 5/2) * sqrt(pi*etax*etay) * gamma(1/4 - (E - en(i, etax) - en(j, etay))/2) *\
                np.exp((E - en(i, etax) - en(j, etay) - 3/2)*Lambda) / (gamma((1-i)/2)**2 * gamma((1-j)/2)**2 *\
                gamma(1+i) * gamma(1+j) * gamma(5/4 - (E - en(i, etax) - en(j, etay))/2)) * sqrt(np.exp(2*Lambda) - 1) *\
                hyp2f1(1, 3/4 - (E - en(i, etax) - en(j, etay))/2, 5/4 - (E - en(i, etax) - en(j, eta_y))/2, np.exp(-2*Lambda))
    return (-1) * suma

def B2_3D(nx, ny, nz, etax, etay, etaz, E, Lambda):
    suma = 0
    for i in range(0, nx+1):
        for j in range(0, ny+1):
            if i%2==0 and j%2==0 and en(i, etax) + en(j, etay) + 1/2 <= E:
                suma += 2**(i + j - 1) * (np.exp(2*Lambda) - 1) * hyp2f1(1, 3/4 - (E - en(i, etax) - en(j, etay))/2,                5/4 - (E - en(i, etax) - en(j, etay))/2, np.exp(-2*Lambda)) / (gamma((1-i)/2)**2 * gamma((1-j)/2)**2 * gamma(1+i) * gamma(1+j) * (2*(E - en(i, etax) - en(j, etay)) - 1))
    return sqrt(pi*etax*etay/np.sinh(Lambda)) * suma

In [10]:
# Integration
h        = 1e-6
a        = h*1e-3 
b        = h*1e4

In [11]:
integral = 0
for i in range(0, 7):
    print(f'''
          {i+1}th integral
          ----------------
          a:    {2*a}
          b:    {2*b}   
          step: {h*10**(-3+i)}        
          N:    {int((2*b-2*a)/(h*10**(-3+i)))}
    ''')
    integral += A3D_int(a, b, h*10**(-3+i), eta_x, eta_y, eta_z, E)
    print('\nDone!\n')
    a = b
    b *= 10
print(10*'-' + '\nBingo !' )
a3D = 4*pi*(integral + B1_3D(nx, ny, nz, eta_x, eta_y, eta_z, E, Lambda) + B2_3D(nx, ny, nz, eta_x, eta_y, eta_z, E, Lambda) + \
            (1/(2*pi))**3/2 * 1/sqrt(2*Lambda))
print(f"""     
               Results
        ---------------------
        asc/dy: {a3D}
""")


          1th integral
          ----------------
          a:    2e-09
          b:    0.02   
          step: 1e-09        
          N:    19999998
    


KeyboardInterrupt: 